# Project CC-DP3 

In this notebook we will show how gesture control can be used for sending commands to an agent, the agent will eventually execute an order related to this gesture.

In this example we use an external webcam to record the livestream of given gestures and will display an animation in real-time to confirm the action related to the gesture.

The gestures are as follows: 

    - FOLLOW: 
    - STOP:
    - (Optional) FETCH: 
